In [127]:
import pandas as pd
import sklearn.tree as sktree
import scipy as sp
import os
import datatable as dt
path = "./데이터/"
filename = os.listdir(path)

In [128]:
target = pd.read_csv(f"{path}침수지역2022.csv")

In [129]:
print("한번이라도 침수가 일어난 지역수(행정동) :", target["ADM_NM"].nunique())

한번이라도 침수가 일어난 지역수(행정동) : 277


#### 침수가 일어난 지역(행정동)

In [130]:
target["ADM_NM"].unique()

array(['우이동', '영등포동', '송중동', '묵1동', '대림1동', '신길6동', '신대방1동', '신길5동',
       '방학2동', '한강로동', '방학1동', '도림동', '신길3동', '회현동', '번1동', '상계5동', '장충동',
       '필동', '서초3동', '다산동', '수유3동', '동화동', '약수동', '상계3·4동', '길동', '방배2동',
       '장위3동', '보광동', '인수동', '남영동', '쌍문3동', '역삼1동', '도곡1동', '반포2동',
       '방배본동', '성수1가1동', '고덕1동', '장위1동', '성수2가1동', '상일동', '삼양동', '장안1동',
       '장안2동', '회기동', '진관동', '용신동', '전농1동', '제기동', '이문1동', '답십리1동',
       '면목본동', '면목3·8동', '면목2동', '면목7동', '방배1동', '상봉1동', '방배3동', '면목4동',
       '대림2동', '구로3동', '조원동', '서초1동', '망우본동', '망우3동', '신내1동', '대림3동',
       '돈암1동', '삼선동', '남가좌2동', '동선동', '월곡1동', '정릉3동', '공덕동', '종암동',
       '개포1동', '장위2동', '반포1동', '충현동', '석관동', '방배4동', '영등포본동', '정릉2동',
       '천호3동', '서초4동', '문래동', '성내2동', '창5동', '미아동', '송천동', '수유2동', '사당1동',
       '번2동', '수유1동', '도봉1동', '쌍문2동', '도봉2동', '쌍문1동', '창1동', '창2동',
       '쌍문4동', '창3동', '방학3동', '월계2동', '공릉1동', '상계1동', '청파동', '상계2동',
       '상계10동', '중화1동', '중계본동', '월계3동', '공항동', '반포4동', '불광2동', '응암3동',
       '성

In [131]:
경사도 = pd.read_excel(path + "/경사도.xlsx")
도로율 = pd.read_csv(path + "/" + "도로현황(도로율)_20231015181102.csv")
제방암거수문 = pd.read_csv(path + "/" + "제방암거수문_있다없다.csv")
배수등급양호비율 = pd.read_csv(path + "/" + "배수등급양호비율.csv")
하천까지의거리 = pd.read_csv(path + "/" + "하천까지의_거리.csv")
인구밀도 = pd.read_csv(path + "/" + "서울시_행정동별_인구밀도_2023.csv")
빗물펌프장까지의거리 = pd.read_excel(path + "/" + "거리계산.xlsx")
월평균소비 = pd.read_csv(path + "/"+ "서울시 상권분석서비스(소득소비-행정동)_20231101_서울열린데이터광장.csv", encoding = 'euc-kr')
관공서합계 = pd.read_excel(path + "/" + "인프라_행정동별_관공서합계(경찰서,병원,소방서).xlsx")
행정동별30년이상건물개수 = pd.read_csv(path + "/" + "행정동별30년이상건물개수.csv")

#### 사회인프라

In [132]:
장애인 = pd.read_csv(path + "/" + "장애인.csv")
노인 = pd.read_csv(path + "/" + "노인.csv")
기초수급자 = pd.read_csv(path + "/" + "기초수급자.csv")
기초수급자_65세이상 = pd.read_csv(path + "/" + "기초생활수급자+65세이상.csv")

#### 강우량

In [133]:
path3 = path + "서울시_강우량_정보_2022년/"

In [134]:
rain_dict = {}
for i in range(1, 13):
    rain_dict[i] = pd.read_csv(path3 + f"서울시_강우량_정보_2022년{i}월.csv", encoding="euc-kr")

In [135]:
rain_data = pd.concat([rain_dict[i][rain_dict[i]["10분우량"] != 0] for i in range(1, 13)])
rain_agg = rain_data.groupby(["구청명"])["10분우량"].agg(["sum", "count"])
rain_agg.rename(
    columns=dict(zip(["sum", "count"], ["누적강수량", "10분당 측정(횟수)"])), inplace=True
)
rain_agg.index.rename(None, inplace=True)

In [136]:
rain_agg["추정시우량(연간)"] = rain_agg["누적강수량"] / (rain_agg["10분당 측정(횟수)"] / 6)

In [137]:
rain_agg.index.rename("시군구", inplace=True)
rain_agg.reset_index(inplace=True)

In [138]:
구면적 = pd.DataFrame(인구밀도.groupby(["구"])["면적 (k㎡)"].sum()).reset_index()
구면적.rename(columns={"구": "시군구"}, inplace=True)

In [139]:
rain_agg = pd.merge(rain_agg, 구면적)
rain_agg["추정면적당시우량(연간)"] = rain_agg["추정시우량(연간)"] / rain_agg["면적 (k㎡)"]

In [140]:
rain_agg

,시군구,누적강수량,10분당 측정(횟수),추정시우량(연간),면적 (k㎡),추정면적당시우량(연간)
0,강남구,5187.0,3025,10.288264,39.51,0.260396
1,강동구,3426.5,2300,8.938696,24.59,0.363509
2,강북구,1803.5,1382,7.829957,23.60,0.331778
3,강서구,2690.5,2053,7.863127,41.44,0.189747
4,관악구,3461.0,2176,9.543199,29.57,0.322732
5,광진구,1887.0,1353,8.368071,17.06,0.490508
6,구로구,3246.5,1962,9.928135,20.12,0.493446
7,금천구,3270.0,2091,9.383070,13.02,0.720666
8,노원구,3290.5,2312,8.539360,35.44,0.240953
9,도봉구,1932.0,1435,8.078049,20.65,0.391189


### 데이터 병합

행정동 단위
> 경사도, 행정공시지가, 배수등급양호비율, 하천까지의 거리, 제방수문암거

In [141]:
import PublicDataReader as pdr

행정동코드 = pdr.code_hdong()
import numpy as np

행정동 = 행정동코드[행정동코드["시도명"] == "서울특별시"][["시군구명", "읍면동명"]]
행정동.index = 행정동.index.astype("int")
행정동["시군구명"] = 행정동["시군구명"].str.strip()
행정동["읍면동명"] = 행정동["읍면동명"].str.strip()
행정동 = 행정동[행정동["시군구명"] != ""][행정동["읍면동명"] != ""]

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023.10.30. 시행)


C:\Users\yis82\AppData\Local\Temp\ipykernel_12612\1195263744.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  행정동 = 행정동[행정동["시군구명"] != ""][행정동["읍면동명"] != ""]


In [142]:
per_경사도 = pd.DataFrame(경사도.groupby(["ADM_NM"])["경사도"].mean())
per_경사도.reset_index(inplace=True)

In [143]:
pre_경사도 = pd.merge(경사도[["GU_NAME", "ADM_NM"]], per_경사도, how="left")
pre_경사도["경사도(%)"] = pre_경사도["경사도"] * 100
pre_경사도.drop_duplicates(["ADM_NM"], inplace=True)

In [144]:
행정동별_녹지면적 = pd.read_csv("./데이터/행정동별 녹지면적.csv")
행정동별_녹지면적

,ADM_NM,GREEN_AREA
0,가락1동,39.607754
1,가락2동,52.844042
2,가락본동,91.714215
3,가리봉동,5.368919
4,가산동,4.972897
...,...,...
418,효창동,136.136473
419,후암동,283.456489
420,휘경1동,15.093137
421,휘경2동,108.279326


In [145]:
X = pd.read_excel(path + "/" + "행정동단위통합데이터.xlsx")

In [146]:
X = X.drop(X.columns[0], axis=1)

In [147]:
X

,Unnamed: 0,GU_NAME,ADM_NM,배수등급양호비율,제방,수문,평균경사도(%),하천까지의거리(중심),인구밀도(명/km²),1분위소득가구비율(%),...,빗물펌프장까지의거리(m),토출량(㎥/분),펌프(합산),노후건물수,월_평균_소득_금액,지출_총금액,경찰,소방서,병원,침수횟수
0,0,송파구,가락1동,0.654167,1,0,3.812500,542.994437,20605.22388,6.976744,...,1336.451802,150,500,14,4973588,5445589750,0,1,6,2
1,2,송파구,가락2동,0.842105,0,0,7.125000,1074.589206,32888.54167,12.245310,...,646.717524,1992,5800,494,3910198,6054228000,0,0,37,11
2,3,송파구,가락본동,0.680000,0,0,10.785714,2056.158356,22821.23894,12.462591,...,358.128508,1992,5800,657,3521561,8160591000,1,0,86,15
3,4,구로구,가리봉동,0.688073,0,0,9.666667,1184.743571,31655.00000,39.355304,...,1007.478249,620,1550,910,2336331,1087078000,1,0,12,19
4,5,금천구,가산동,0.610245,1,0,8.772727,468.909180,10567.85714,22.982635,...,466.990492,640,1590,788,2567594,36128649250,1,0,65,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,815,용산구,효창동,0.980769,1,0,12.666667,3363.536779,24777.27273,14.144849,...,1284.697842,1096,3100,419,3523201,974093250,0,0,12,0
396,817,용산구,후암동,1.000000,1,0,39.625000,346.532474,19465.11628,21.626764,...,1819.097470,630,1400,1177,3013677,2002563750,1,1,15,0
397,819,동대문구,휘경1동,0.405797,1,0,5.800000,754.989114,25936.50794,32.527147,...,357.712372,668,2280,1200,2722774,3357385250,0,0,28,0
398,821,동대문구,휘경2동,0.300578,1,1,5.583333,640.702435,24180.95238,26.526171,...,300.317946,668,2280,887,3071586,2217352000,1,1,8,0


In [148]:
X["행정동별_녹지면적(km²)"] = 행정동별_녹지면적['GREEN_AREA']

In [149]:
X

,Unnamed: 0,GU_NAME,ADM_NM,배수등급양호비율,제방,수문,평균경사도(%),하천까지의거리(중심),인구밀도(명/km²),1분위소득가구비율(%),...,토출량(㎥/분),펌프(합산),노후건물수,월_평균_소득_금액,지출_총금액,경찰,소방서,병원,침수횟수,행정동별_녹지면적(km²)
0,0,송파구,가락1동,0.654167,1,0,3.812500,542.994437,20605.22388,6.976744,...,150,500,14,4973588,5445589750,0,1,6,2,39.607754
1,2,송파구,가락2동,0.842105,0,0,7.125000,1074.589206,32888.54167,12.245310,...,1992,5800,494,3910198,6054228000,0,0,37,11,52.844042
2,3,송파구,가락본동,0.680000,0,0,10.785714,2056.158356,22821.23894,12.462591,...,1992,5800,657,3521561,8160591000,1,0,86,15,91.714215
3,4,구로구,가리봉동,0.688073,0,0,9.666667,1184.743571,31655.00000,39.355304,...,620,1550,910,2336331,1087078000,1,0,12,19,5.368919
4,5,금천구,가산동,0.610245,1,0,8.772727,468.909180,10567.85714,22.982635,...,640,1590,788,2567594,36128649250,1,0,65,56,4.972897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,815,용산구,효창동,0.980769,1,0,12.666667,3363.536779,24777.27273,14.144849,...,1096,3100,419,3523201,974093250,0,0,12,0,137.779560
396,817,용산구,후암동,1.000000,1,0,39.625000,346.532474,19465.11628,21.626764,...,630,1400,1177,3013677,2002563750,1,1,15,0,825.793557
397,819,동대문구,휘경1동,0.405797,1,0,5.800000,754.989114,25936.50794,32.527147,...,668,2280,1200,2722774,3357385250,0,0,28,0,141.544094
398,821,동대문구,휘경2동,0.300578,1,1,5.583333,640.702435,24180.95238,26.526171,...,668,2280,887,3071586,2217352000,1,1,8,0,34.253825


In [150]:
도로 = pd.read_csv("./데이터/도로.csv")
건물 = pd.read_csv("./데이터/건물.csv")

In [151]:
건물 = 건물[["ADM_NM",'SHAPE_AREA']]

In [152]:
도로 = 도로[["ADM_NM",'ROAD_BT', "ROAD_LT"]]
도로['ROAD_AREA']  = 도로['ROAD_BT'] * 도로['ROAD_LT']

In [153]:
도로 = 도로.groupby('ADM_NM').sum()

In [158]:
건물 = 건물.groupby('ADM_NM').sum()

,SHAPE_AREA
ADM_NM,
가락1동,341973.83
가락2동,219508.10
가락본동,253786.39
가리봉동,168897.44
가산동,817992.38
...,...
효창동,119502.65
후암동,237745.33
휘경1동,161263.19


In [159]:
도로_건물 = pd.merge(도로, 건물, on="ADM_NM")

In [161]:
도로_건물['도로_건물_면적'] = 도로_건물['ROAD_AREA'] + 도로_건물['SHAPE_AREA']

In [163]:
도로_건물.to_csv('./데이터/도로_건물.csv')